In [97]:
word2id,id2word = {},{}
tag2id,id2tag = {},{}
for line in open("traindata.txt"):
    pro_line = line.split("/")
    word = pro_line[0]
    tag = pro_line[1].rstrip()
    if word not in word2id:
        word2id[word] = len(word2id)
        id2word[len(id2word)] = word
    if tag not in tag2id:
        tag2id[tag] = len(tag2id)
        id2tag[len(id2tag)] = tag

M = len(word2id) #number of words
N = len(tag2id)  #number of tags


18978


In [109]:
import numpy as np
A = np.zeros((N,M))
pi = np.zeros(N)
B = np.zeros((N,N))
pre_tag = ""
for line in open("traindata.txt"):
    items = line.split("/")
    wordID = word2id[items[0]]
    tagID = tag2id[items[1].rstrip()]
    if pre_tag == "":
        pi[tagID] += 1
        A[tagID][wordID] += 1
    else:
        A[tagID][wordID] += 1
        B[tag2id[pre_tag]][tagID] += 1
        
    if items[0] == ".":
        pre_tag = ""
    else:
        pre_tag = items[1].rstrip()

pi = pi/sum(pi)
for i in range(N):
    A[i] = A[i]/sum(A[i])
    B[i] = B[i]/sum(B[i])


In [99]:
def log(x):
    if x == 0:
        return np.log(x+0.000001)
    return np.log(x)

In [108]:
def main(x):
    sentences = [word2id[word] for word in x.split(" ")]
    sent_len = len(sentences)
    
    D = np.zeros((sent_len,N))
    position = np.array([[0 for x in range(N)] for y in range(sent_len)])
    
    for p in range(N):
        D[0][p] = log(pi[p]) + log(A[p][sentences[0]])
        
    for i in range(1,sent_len):
        for j in range(N):
            D[i][j] = -9999999
            for k in range(N):
                score = D[i-1][k] + log(A[j][sentences[i]]) + log(B[k][j])
                if score > D[i][j]:
                    D[i][j] = score
                    position[i][j] = k
    
    best_seq = [0]*sent_len
    best_seq[sent_len-1] = np.argmax(D[sent_len-1])
    
    for i in range(sent_len-2,-1,-1):
        best_seq[i] = position[i+1][best_seq[i+1]]
    
    for i in best_seq:
        print(id2tag[i])
        
a = "I study hard"
main(a)

PRP
VBP
RB
